# Correlaties — Stappenteller 2024–2025 (STUDENTENVERSIE)

In deze opdracht onderzoek je **samenhang** tussen variabelen: hangen twee variabelen met elkaar samen?

We bekijken de relatie tussen:

- stappen op **dag 1**
- en het **totaal aantal stappen over de week**

Daarna maken we het inhoudelijk sterker door dag 1 uit het weektotaal te halen.

## Leerdoelen

Na deze opdracht kun je:

- een scatterplot interpreteren
- Pearson correlatie \(r\) berekenen en uitleggen
- (optioneel) Spearman correlatie uitleggen
- begrijpen dat correlatie **geen causaliteit** bewijst
- herkennen wanneer een correlatie “automatisch” ontstaat door hoe variabelen zijn opgebouwd

## Korte theorie: wat is correlatie?

Een correlatiecoëfficiënt beschrijft de **sterkte** en **richting** van een (meestal lineaire) samenhang.

Voor Pearson correlatie \(r\):

- \(r = 1\): perfecte positieve lineaire samenhang  
- \(r = 0\): geen lineaire samenhang  
- \(r = -1\): perfecte negatieve lineaire samenhang  

Vuistregels (heel grof, context-afhankelijk):
- |r| ≈ 0.1: zwak  
- |r| ≈ 0.3: matig  
- |r| ≥ 0.5: sterk  

> Correlatie zegt niets over oorzaak-gevolg: **correlatie ≠ causaliteit**.

# 1. Data inladen

We laden dezelfde dataset in als in opdracht 1. Daarna maken we twee variabelen:

- `dag1` = stappen op dag 1  
- `stapWeek` = som van dag 1 t/m 7

Let op: er kunnen missende waarden (`NA`) voorkomen.

In [ ]:
df <- read.csv("https://raw.githubusercontent.com/DannyLemmers/Stappenteller-2025/main/data/StappentellerData.csv")

# (optioneel) verwijder lege geslacht-waarden als je dat eerder ook deed:
df <- df[df$Geslacht != "", ]

# Maak stapWeek (som van 7 dagen)
df$stapWeek <- df$`StapAantal.1.` + df$`StapAantal.2.` + df$`StapAantal.3.` +
              df$`StapAantal.4.` + df$`StapAantal.5.` + df$`StapAantal.6.` +
              df$`StapAantal.7.`

# Dag 1 apart
df$dag1 <- df$`StapAantal.1.`

# Verwijder rijen met NA in dag1 of stapWeek
df <- df[!is.na(df$dag1) & !is.na(df$stapWeek), ]

# Snelle check
summary(df[, c("dag1", "stapWeek")])

# 2. Eerste analyse: dag 1 vs weektotaal

Onderzoeksvraag:

> Bestaat er een samenhang tussen stappen op dag 1 en het totaal aantal stappen over de week?

Eerste stap: maak een **scatterplot** (puntenwolk).

In [ ]:
plot(df$dag1, df$stapWeek,
     xlab = "Stappen op dag 1",
     ylab = "Totaal stappen per week",
     main = "Dag 1 vs weektotaal")

# Regressielijn (helpt bij het zien van lineaire trend)
abline(lm(stapWeek ~ dag1, data = df), col = "red")

### Vragen (scatterplot)

- Zie je een stijgende trend (meer dag 1 → meer weektotaal)?  
  - …

- Vind je de puntenwolk “strak” (sterke relatie) of “breed” (zwakke relatie)?  
  - …

- Zijn er duidelijke outliers die de relatie kunnen beïnvloeden?  
  - …

## 2.1 Pearson correlatie berekenen

We berekenen Pearson \(r\) en voeren een toets uit met `cor.test()`.

De toets gebruikt:

- H0: correlatie = 0 (geen lineaire samenhang)
- H1: correlatie ≠ 0 (wel lineaire samenhang)

Let op: bij grote steekproeven kan een kleine correlatie toch “significant” zijn.
Daarom interpreteren we **altijd** ook de grootte van \(r\).

In [ ]:
# Correlatiecoëfficiënt
r_pearson <- cor(df$dag1, df$stapWeek, use = "complete.obs", method = "pearson")
r_pearson

# Correlatietoets
test_pearson <- cor.test(df$dag1, df$stapWeek, method = "pearson")
test_pearson

### Vragen (Pearson)

- Wat is de waarde van \(r\)?  
  - …

- Is de correlatie positief of negatief?  
  - …

- Is de correlatie zwak/matig/sterk? (onderbouw kort)  
  - …

- Wat is de p-waarde? Wat concludeer je bij α = 0.05?  
  - …

- Schrijf één zin die \(r\) interpreteert in context:  
  - …

# 3. Kritisch denken: deze correlatie is deels “automatisch”

Belangrijk punt:

`stapWeek` is gedefinieerd als:

> stapWeek = dag1 + dag2 + dag3 + dag4 + dag5 + dag6 + dag7

Dus dag 1 zit **in** het weektotaal.  
Daardoor zal er bijna altijd een **positieve** correlatie zijn, zelfs als dag 2 t/m 7 weinig met dag 1 te maken hebben.

Dit betekent niet dat de analyse “fout” is, maar de interpretatie moet voorzichtig zijn.

Een inhoudelijk interessantere vraag is:

> Als iemand op dag 1 veel loopt, loopt die persoon dan ook op de **andere** dagen veel?

# 4. Sterkere analyse: dag 1 vs (week zonder dag 1)

We maken een nieuwe variabele:

- `weekZonderDag1` = stapWeek − dag1

Nu testen we of dag 1 samenhangt met de rest van de week.

In [ ]:
df$weekZonderDag1 <- df$stapWeek - df$dag1

summary(df[, c("dag1", "weekZonderDag1")])

In [ ]:
plot(df$dag1, df$weekZonderDag1,
     xlab = "Stappen op dag 1",
     ylab = "Stappen in de rest van de week (zonder dag 1)",
     main = "Dag 1 vs rest van de week")

abline(lm(weekZonderDag1 ~ dag1, data = df), col = "red")

### Vragen (scatterplot 2)

- Is de relatie nog steeds duidelijk?  
  - …

- Is deze relatie sterker/zwakker dan bij dag1 vs stapWeek?  
  - …

- Wat zegt dat over “consistentie” van stappen over de week?  
  - …

In [ ]:
r2_pearson <- cor(df$dag1, df$weekZonderDag1, use = "complete.obs", method = "pearson")
r2_pearson

test2_pearson <- cor.test(df$dag1, df$weekZonderDag1, method = "pearson")
test2_pearson

### Vragen (Pearson 2)

- Wat is \(r\) voor dag1 vs weekZonderDag1?  
  - …

- Wat concludeer je bij α = 0.05?  
  - …

- Wat is inhoudelijk jouw conclusie over deze relatie?  
  - …

# 5. (Optioneel) Spearman correlatie

Pearson kijkt naar **lineaire** samenhang en is gevoelig voor outliers.

Spearman correlatie (rho) gebruikt rangen en is robuuster bij:
- niet-lineaire maar monotone relaties
- outliers
- scheve verdelingen

We berekenen Spearman voor beide relaties en vergelijken.

In [ ]:
# Spearman voor dag1 vs stapWeek
spearman1 <- cor.test(df$dag1, df$stapWeek, method = "spearman")
spearman1

# Spearman voor dag1 vs weekZonderDag1
spearman2 <- cor.test(df$dag1, df$weekZonderDag1, method = "spearman")
spearman2

### Vragen (Spearman, optioneel)

- Zijn de Spearman-resultaten vergelijkbaar met Pearson?  
  - …

- Welke correlatie zou jij rapporteren en waarom?  
  - …

# 6. Eindconclusie

Beantwoord in 2–4 zinnen:

1. Hoe groot is de samenhang tussen dag 1 en het weektotaal?  
2. Waarom is deze correlatie deels “automatisch”?  
3. Wat is je conclusie voor dag 1 vs de rest van de week?

**Antwoord:**  
- …

## 6.1 Rapportagezin (voorbeeldstructuur)

Schrijf één zin die je in een verslag zou zetten, bijvoorbeeld:

> “Er was een [zwakke/matige/sterke] positieve correlatie tussen … (Pearson r = …, p = …).”

**Jouw rapportagezin:**  
- …